# Optuna pipeline

Цель данного ноутбука -- показать, как можно применять библиотеку `optuna` для оптимизации гиперпараметров и подбора элементов пайплайна. Начало скопировано из ноутбука про `hyperopt`.

Главная [страница](https://optuna.org/) библиотеки. [Документация](https://optuna.readthedocs.io/en/stable/index.html).
Судя по документации, это довольно интересная штука с большими возможностями по настройке, сохранению и обработке результатов экспериментов.
Интересные страницы, которые стоит посмотреть:
 - [optuna.study](https://optuna.readthedocs.io/en/stable/reference/study.html)
 - [Study](https://optuna.readthedocs.io/en/stable/reference/generated/optuna.study.Study.html#optuna.study.Study)
 - [Trial](https://optuna.readthedocs.io/en/stable/reference/generated/optuna.trial.Trial.html#optuna.trial.Trial)
 - [как настроить verbose](https://optuna.readthedocs.io/en/stable/reference/logging.html)

Далее идет кусок из ноута по аналогии с ноутбуком по `hyperopt`.

---

## 1. Technicals

In [1]:
import time
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import optuna
import umap
seed=42

from sklearn.metrics import roc_auc_score
from imblearn.pipeline import Pipeline

# from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler


# новый пакет!
from feature_engine.encoding import WoEEncoder
from feature_engine.creation import CombineWithReferenceFeature
from feature_engine.encoding import OneHotEncoder

from feature_engine.encoding.base_encoder import BaseCategoricalTransformer
from feature_engine.validation import _return_tags
from feature_engine.variable_manipulation import _check_input_parameter_variables

from mlxtend.feature_selection import SequentialFeatureSelector
from feature_engine.selection  import SelectByShuffling
from feature_engine.selection  import RecursiveFeatureAddition
from feature_engine.selection  import SmartCorrelatedSelection

## 2. Import Dataset

In [2]:
X_train = pd.read_parquet('../datasets/01_german/samples/X_train.parquet')
y_train = pd.read_parquet('../datasets/01_german/samples/y_train.parquet').target

# X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.33, random_state=42)

X_test  = pd.read_parquet('../datasets/01_german/samples/X_test.parquet')
y_test  = pd.read_parquet('../datasets/01_german/samples/y_test.parquet').target

In [3]:
with open('../datasets/01_german/factors.json') as json_file:
    factors_dict = json.load(json_file)

factors_dict['cat_vals']

['cheq_acc',
 'cred_hist',
 'purp',
 'save_acc',
 'empl_t',
 'pers_status',
 'guarant_flg',
 'prop',
 'inst_plan',
 'house',
 'job',
 'tel_flg',
 'foreign_flg']

## 3. Define Modules

All the modules that might be part of the pipeline should be defined below (or import them):

In [4]:
# костыль для импорта из "соседней" директории
import sys
sys.path.insert(1, "../")

from modules.feature_selection import CombineWithReferenceFeature_adj
from modules.preprocessing import DimensionReducer
from modules.encoders import WoEEncoder_adj

In [5]:
WoE_module = WoEEncoder_adj(variables = factors_dict['cat_vals'])

OneHot_module = OneHotEncoder(variables = factors_dict['cat_vals'])

PCA_module = DimensionReducer(
    gen_class = sklearn.decomposition.PCA,
    n_components = 2,    # сколько оставить компонентов; по дефолту - все
    whiten = False,      # отключаем whitening - декорреляцию фичей
    svd_solver = "full", # детали SVD преобразования, за подробностями см. доки
)

kPCA_module = DimensionReducer(
    gen_class = sklearn.decomposition.KernelPCA,
    n_components = 8,  # сколько оставить компонентов; по дефолту - все
    kernel = "linear", # ядро. По дфеолту линейное. Можно сделать своё, но тогда его нужно предварительно вычислить отдельно,
                       # поставить kernel = "precomputed" и передать уже вычисленное ядро в качестве X
    degree = 3,        # степень полинома для некоторых типов ядер. Важный параметр для тьюнинга, но сильно напрягает процессор
    n_jobs = -1        # объект умеет быть многопоточным! -1 займет все ядра
)

Isomap_module = DimensionReducer(
    gen_class = sklearn.manifold.Isomap,
    n_neighbors = 5, #количество соседей при вычислении KNN. Основной гиперпараметр, кстати (!!!)
    n_components = 2,  #сколько оставить компонент; по дефолту - 2
    path_method = "auto", #алгоритм, который вычисляет кратчайший путь. Варианты см. на странице функции. Этот подбирает сам.
    neighbors_algorithm = "auto", #алгоритм, который ищет соседей. Инстанс класса NearestNeighbours
    n_jobs = -1 #объект умеет быть многопоточным! -1 займет все ядра
)

UMAP_module = DimensionReducer(
    gen_class = umap.UMAP,
    n_neighbors = 5,  # количество соседей при вычислении KNN. Основной гиперпараметр, кстати (!!!)
    n_components = 2, # сколько оставить компонентов; по дефолту - 2
    min_dist = 0.1    # минимальная дистанция, которую можно сохранять между точками в получающемся пространстве. 
    # Гиперпараметр. При увеличении начинает лучше улавливать общую структуру, но хуже - локальную
)

CombWRef_module = CombineWithReferenceFeature_adj(
    operations = ['mul']
)

lgbm_mdl = LGBMClassifier(
    num_leaves = 10,
    learning_rate = .1,
    reg_alpha = 8,
    reg_lambda = 8,
    random_state = seed
)

# Tackling imbalances in target
RUS_module    = RandomUnderSampler(random_state = seed)
ROS_module    = RandomOverSampler(random_state = seed)
SMOTE_module  = SMOTE(random_state = seed)
ADASYN_module = ADASYN(random_state = seed)

# feature selection
SeqFearSel_module = SequentialFeatureSelector(
    estimator  = lgbm_mdl,  
    # k_features = 5,                                                  
    forward    = True,                                                  
    floating   = True,                                                
    verbose    = 0,
    cv         = 5
)
RecFeatAdd_module = RecursiveFeatureAddition(
    lgbm_mdl,
    threshold = 0.005
)
# SelShuffl_module = SelectByShuffling(
#     estimator = lgbm_mdl,
#     # variables=X.columns.to_list(),                                      # можно задать подмножество
#     scoring='roc_auc',                                                  # метрика
#     threshold=0.01,                                                     # порог ее снижения
#     cv=5,
#     random_state=42
# )
SmartSel_module = SmartCorrelatedSelection(
    # variables=X.columns.to_list(),
    method="pearson",                # можно взять свою функцию
    threshold=0.3,                   # порог корреляции
    selection_method="variance",     # из коррелирующих групп выбираем признак с наиб дисперсией
    estimator=None,                  # понадобится для selection_method="model_performance"        
    cv=5
)


In [6]:
modules = {
    'WoE':         WoE_module,
    'OneHot':      OneHot_module,
    'PCA':         PCA_module,
    'kPCA':        kPCA_module,
    'Isomap':      Isomap_module,
    'UMAP':        UMAP_module,
    'CombWRef':    CombWRef_module,
    'RecFeatAdd':  RecFeatAdd_module,
    'lgbm':        lgbm_mdl,
    'RUS':         RUS_module,      
    'ROS':         ROS_module,      
    'SMOTE':       SMOTE_module,  
    'ADASYN':      ADASYN_module,
    'SeqFearSel':  SeqFearSel_module,
    'RecFeatAdd':  RecFeatAdd_module,
    # 'SelShuffl':   SelShuffl_module,
    'SmartSel':    SmartSel_module,
    'passthrough' : 'passthrough'      # можно добавить в (не последнюю) ячейку пайплайна, чтобы пропустить ее
}

## 4. Define Pipeline

Кандидаты для каждого элемента пайплайна (`passthrough` означает, что пропускаем).

In [7]:
pipe_params = {
    # 'missing_vals': 
    'cat_encoding':  ['OneHot', 'WoE'], # , 'woe' пропустить нельзя из-за наличия кат. пер-х
    'imbalance':     ['passthrough', 'RUS', 'ROS', 'SMOTE', 'ADASYN'],
    'feat_eng':      ['passthrough', 'PCA', 'kPCA', 'Isomap', 'UMAP'], # , 'CombWRef' # удалил, т.к. долго считается
    'feat_sel':      ['passthrough', 'SeqFearSel', 'RecFeatAdd', 'SmartSel'], # 'SelShuffl' is omitted, since it might drop all Xs
    'lgbm':          ['lgbm']
}

Функции для выбора гиперпараметров для отдельных элементов -- самая напряжная часть оптуны, т.к. нужно вручную прописывать, из какого множества выбирать:

In [8]:
def PCA_params(trial) -> dict:
    return {
        "n_components" : trial.suggest_int("PCA__n_components", 2, 11),
        "whiten" : trial.suggest_categorical(
            "PCA__whiten",
            [True, False]
            ),
        "svd_solver" : trial.suggest_categorical(
            "PCA__svd_solver",
            ['full', 'arpack', 'auto', 'randomized']
            )
    }

def kPCA_params(trial):
    return {
        "n_components" : trial.suggest_int("kPCA__n_components", 5, 11),
        "kernel" : trial.suggest_categorical(
            "kPCA__kernel",
            ['linear', 'poly', 'rbf', 'sigmoid', 'cosine', 'precomputed']
            ),
    }

def Isomap_params(trial):
    return {
        "n_neighbors" : trial.suggest_int("Isomap__n_neighbors", 2, 11),
        "n_components" : trial.suggest_int("Isomap__n_components", 2, 5),
        "path_method" : trial.suggest_categorical(
            "Isomap__path_method",
            ['auto', 'FW', 'D']
            )
    }

def UMAP_params(trial):
    return {
        "n_neighbors" : trial.suggest_int("UMAP__n_neighbors", 2, 11),
        "n_components" : trial.suggest_int("UMAP__n_components", 2, 11),
        "min_dist" : trial.suggest_float("UMAP__min_dist", 0.05, 1, step=0.05)
    }

def LightGBM_params(trial):
    return {
        "learning_rate" : trial.suggest_float("LightGBM__learning_rate", 0.05, 0.31, step=0.05),
        "num_leaves" : trial.suggest_int("LightGBM__num_leaves", 5, 16),
        "reg_alpha" : trial.suggest_int("LightGBM__reg_alpha", 0, 16),
        "reg_lambda" : trial.suggest_int("LightGBM__reg_lambda", 0, 0.16),
        "n_estimators" : 100
    }

Сложим их в словарик и подадим его на вход оптимизируемой функции:

In [9]:
modules_hparams = {
    'PCA' : PCA_params,
    'kPCA' : kPCA_params,
    'UMAP' : UMAP_params,
    'LightGBM' : LightGBM_params
}

Оптимизиуемая функция. Должна принимать на вход объект `trial` и возвращать значение метрики:

In [10]:
def Gini(y, y_pred):
    res = roc_auc_score(y, y_pred) * 2 - 1
    print(f"Gini: {res}")
    return(res)

def objective(trial, modules, pipe_params, modules_hparams) -> float:
    """
    Функция для подачи в библиотеку.

    Parameters
    ----------
    trial : ...
        Вспомогательный аргумент, используется непосредственно оптуной.
    modules : dict
        Ключ - название модуля, значение - объект данного класса.
        Примеры пар: 'WoE' : WoE_module, "passthrough" : "passthrough".
    pipe_params : dict
        Ключ - этап пайплайна, значение - лист с вариантами на данный этап.
        Пример пары: 'feat_eng' : ['passthrough', 'PCA', 'kPCA', 'Isomap', 'UMAP'].
    modules_params : dict
        Ключ - название модуля (как у modules),
        значение - функция: trial -> dict с параметрами данного модуля

    Returns
    -------
    Значение оптимизируемой метрики (в данном случае gini).
    """
    pipeline = []
    
    # часть 1: выбираем модули и параметры к ним
    for elem, options in pipe_params.items():                                    # итерация по парам (ключ, значение) словаря
        choice : str = trial.suggest_categorical(f"pipe__{elem}", options)       # выбираем элемент пайплайна
        module = modules[choice]                                                 # достаем этот объект из словаря

        if choice in modules_hparams.keys():                                     # если мы также оптимизируем гиперпараметры для элемента пайплайна
            hp_function : callable = modules_hparams[choice]                     # функция, которая возвращает гиперпараметры
            hp_choice : dict = hp_function(trial)                                # конкретный набор, который она вернула
            module.set_params(**hp_choice)

        pipeline.append(
            # (choice, module)
            (elem, module)
        )
    
    pipeline = Pipeline(pipeline)                                                # получаем модель

    # часть 2: оцениваем качество получившейся модели
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict_proba(X_test)[:, 1]
    metric = roc_auc_score(y_test, y_pred)

    return metric

Собственно запуск оптимизации:

In [11]:
study = optuna.create_study(
    direction='maximize',
    sampler=optuna.samplers.TPESampler(seed=seed)  # необязательный аргумент, однако seed настраивается через него
                                                   # сид работает, после перезапуска кода результат воспроизводится
    )
study.optimize(
    func=lambda trial: objective(trial, modules, pipe_params, modules_hparams),  # оптимизируемая функция
    n_trials=5,                                                                  # число попыток
    timeout=600,                                                                 # лимит по времени
    )

[I 2022-01-09 23:24:02,629] A new study created in memory with name: no-name-8905eced-b8b9-45d1-882b-160210d0c25a
[I 2022-01-09 23:24:24,611] Trial 0 finished with value: 0.5680971270585405 and parameters: {'pipe__cat_encoding': 'WoE', 'pipe__imbalance': 'passthrough', 'pipe__feat_eng': 'UMAP', 'UMAP__n_neighbors': 10, 'UMAP__n_components': 4, 'UMAP__min_dist': 0.2, 'pipe__feat_sel': 'RecFeatAdd', 'pipe__lgbm': 'lgbm'}. Best is trial 0 with value: 0.5680971270585405.
d:\environments\scoring_project\lib\site-packages\sklearn\manifold\_spectral_embedding.py:260: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(
[I 2022-01-09 23:24:32,022] Trial 1 finished with value: 0.7875946628200505 and parameters: {'pipe__cat_encoding': 'WoE', 'pipe__imbalance': 'ADASYN', 'pipe__feat_eng': 'UMAP', 'UMAP__n_neighbors': 3, 'UMAP__n_components': 2, 'UMAP__min_dist': 0.9500000000000001, 'pipe__feat_sel': 'passthrough', 'pipe__lgbm': 'lgbm'}. Best is 

In [12]:
study.best_params

{'pipe__cat_encoding': 'WoE',
 'pipe__imbalance': 'ADASYN',
 'pipe__feat_eng': 'PCA',
 'PCA__n_components': 3,
 'PCA__whiten': True,
 'PCA__svd_solver': 'full',
 'pipe__feat_sel': 'passthrough',
 'pipe__lgbm': 'lgbm'}

In [13]:
study.best_trial

FrozenTrial(number=3, values=[0.8025003005168889], datetime_start=datetime.datetime(2022, 1, 9, 23, 24, 32, 230810), datetime_complete=datetime.datetime(2022, 1, 9, 23, 24, 32, 439312), params={'pipe__cat_encoding': 'WoE', 'pipe__imbalance': 'ADASYN', 'pipe__feat_eng': 'PCA', 'PCA__n_components': 3, 'PCA__whiten': True, 'PCA__svd_solver': 'full', 'pipe__feat_sel': 'passthrough', 'pipe__lgbm': 'lgbm'}, distributions={'pipe__cat_encoding': CategoricalDistribution(choices=('OneHot', 'WoE')), 'pipe__imbalance': CategoricalDistribution(choices=('passthrough', 'RUS', 'ROS', 'SMOTE', 'ADASYN')), 'pipe__feat_eng': CategoricalDistribution(choices=('passthrough', 'PCA', 'kPCA', 'Isomap', 'UMAP')), 'PCA__n_components': IntUniformDistribution(high=11, low=2, step=1), 'PCA__whiten': CategoricalDistribution(choices=(True, False)), 'PCA__svd_solver': CategoricalDistribution(choices=('full', 'arpack', 'auto', 'randomized')), 'pipe__feat_sel': CategoricalDistribution(choices=('passthrough', 'SeqFearSel

In [14]:
study.best_value

0.8025003005168889